In [1]:
import threading
import requests
import json
import random
import time
import pandas as pd
from queue import Queue
from requests.adapters import HTTPAdapter

In [2]:
def windy_scratch(lat,lng,queue,timeout_lists):
    s = requests.Session()
    s.mount('http://', HTTPAdapter(max_retries=3))
    s.mount('https://', HTTPAdapter(max_retries=3))
    try:
        url = requests.get(f"https://node.windy.com/forecast/point/ecmwf/v2.7/{lat}/{lng}?source=detail&step=3&token=&token2=pending&uid=f02b10d3-e08e-7976-cc71-e63af03a76af&sc=1&pr=1&v=37.2.2&poc=",timeout=5)
        api_data = json.loads(url.text)
        api_data = pd.DataFrame(api_data)
        api_data = api_data.T
        if api_data.iloc[0].isna().any() == True:
            api_data.fillna(method='backfill',inplace=True)
        api_data = api_data.iloc[[0]]
        queue.put(api_data)
    except requests.exceptions.RequestException:
        a = []
        a.append(lat)
        a.append(lng)
        timeout_lists.put(a)
        pass

In [13]:

df = pd.read_csv('all_data.csv')
lat = df['lat']
lng = df['lng']
url = requests.get(f"https://node.windy.com/forecast/point/ecmwf/v2.7/{lat[1531]}/{lng[1531]}?source=detail&step=3&token=&token2=pending&uid=f02b10d3-e08e-7976-cc71-e63af03a76af&sc=1&pr=1&v=37.2.2&poc=",timeout=5)
api_data = json.loads(url.text)
api_data = pd.DataFrame(api_data)
api_data = api_data.T

C:\Users\eric0\AppData\Local\Temp\ipykernel_18100\2772158164.py:1: DtypeWarning: Columns (2,4,7,9,10,11,12,13,14,19,20,21,22,23,24,26,28,31,32,33,34,35,36,37,38,43,44,45,46,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,70,71,72,73,74,76,102,103,114,115,116,117,119,120,122,125,126) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('all_data.csv')


In [3]:
if __name__ == '__main__':
    df = pd.read_csv('all_data.csv',chunksize=30)

    num = 0
    Thread_Group = []
    q_item_url = Queue()
    time_out = Queue()
    for chunk in df:
        num = len(chunk)+num
        print(num)
        for chunk_number in range(len(chunk)):
            lat = chunk['lat'].to_list()[chunk_number]
            lng = chunk['lng'].to_list()[chunk_number]
            t1 = threading.Thread(target = windy_scratch,args=(lat,lng,q_item_url,time_out))
            Thread_Group.append(t1)
        for t in Thread_Group:
            t.start()
        for t in Thread_Group:
            t.join()
        Thread_Group.clear()
        print(Thread_Group)
        delay_choices = [2,3,4]  #延遲的秒數
        delay = random.choice(delay_choices)  #隨機選取秒數
        time.sleep(delay)  #延遲
    
    product_list = []
    for i in range(q_item_url.qsize()):
        product_list.append(q_item_url.get())
    data_df = product_list[0]
    for i in range(1,len(product_list)):
        data_df = pd.concat([data_df,product_list[i]],ignore_index=True,axis=0)
    data_df.to_csv('windy_data.csv')

30
[]
60
[]
90
[]
120
[]
150
[]
180
[]
210
[]
240
[]
270
[]
300
[]
330
[]
360
[]
390
[]
420
[]
450
[]
480
[]
510
[]
540
[]
570
[]
600
[]
630
[]
660
[]
690
[]
720
[]
750
[]
780
[]
810
[]
840
[]
870
[]
900
[]
930
[]
960
[]
990
[]
1020
[]
1050
[]
1080
[]
1110
[]
1140
[]
1170
[]
1200
[]
1230
[]
1260
[]
1290
[]
1320
[]
1350
[]
1380
[]
1410
[]
1440
[]
1470
[]
1500
[]
1530
[]
1560
[]
1590
[]
1620
[]
1650
[]
1680
[]
1710
[]
1740
[]
1770
[]
1800
[]
1830
[]
1860
[]
1890
[]
1920
[]
1950
[]
1980
[]
2010
[]
2040
[]
2070
[]
2100
[]
2130
[]
2160
[]
2190
[]
2220
[]
2250
[]
2280
[]
2310
[]
2340
[]
2370
[]
2400
[]
2430
[]
2460
[]
2490
[]
2520
[]
2550
[]
2580
[]
2610
[]
2640
[]
2670
[]
2700
[]
2730
[]
2760
[]
2790
[]
2820
[]
2850
[]
2880
[]
2910
[]
2940
[]
2970
[]
3000
[]
3030
[]
3060
[]
3090
[]
3120
[]
3150
[]
3180
[]
3210
[]
3240
[]
3270
[]
3300
[]
3330
[]
3360
[]
3390
[]
3420
[]
3450
[]
3480
[]
3510
[]
3540
[]
3570
[]
3600
[]
3630
[]
3660
[]
3690
[]
3720
[]
3750
[]
3780
[]
3810
[]
3840
[]
3870
[]
3900

In [4]:
time_list = []    
for i in range(time_out.qsize()):
    time_list.append(time_out.get())


In [9]:
data_df = pd.DataFrame(data_df)
data_df.to_csv('time_out.csv')

In [8]:
import multiprocessing as mp

mp.cpu_count()

8